#Hackathon - Food delivery data analysis


This project combines transactional, user, and restaurant datasets to create a unified food delivery dataset.
The analysis focuses on:
1. Order trends over time
2. User behavior patterns
3. City-wise and cuisine-wise performance
4. Membership impact (Gold vs Regular)
5. Revenue distribution and seasonality

In [46]:
from google.colab import files
files.upload()
#Load the datasets
import pandas as pd
import sqlite3
orders = pd.read_csv("orders.csv")
users = pd.read_json("users.json")
conn = sqlite3.connect("restaurants.db")
with open("restaurants.sql", "r") as file:
    sql_script = file.read()
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
#Merging the datasets
orders_users = orders.merge(
    users,
    on = "user_id",
    how ="left"
)
final_df = orders_users.merge(
    restaurants,
    on= "restaurant_id",
    how= "left"
)

final_df.to_csv("final_food_delivery_dataset.csv" , index= False)
final_df.columns

Saving orders.csv to orders (6).csv
Saving users.json to users (6).json
Saving restaurants.sql to restaurants (6).sql


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [47]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'],dayfirst= True)
final_df.groupby(final_df['order_date'].dt.month)['order_id'].count()

final_df.groupby('membership')['order_id'].count()

final_df.groupby(['city', 'cuisine'])['total_amount'].sum()

final_df.groupby('membership')['total_amount'].mean()

final_df.groupby(final_df['order_date'].dt.month)['total_amount'].sum()

,total_amount
order_date,
1,663020.26
2,630867.40
3,716738.98
4,666097.18
5,668428.61
6,610822.93
7,688559.45
8,694987.58
9,653838.07


In [48]:
#highest revenue city among gold members
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .sum() \
    .sort_values(ascending=False) \
    .head(1)

,total_amount
city,
Chennai,1080909.79


In [49]:
#cuisine with highest avg order value
final_df.groupby('cuisine')['total_amount'] \
    .mean() \
    .sort_values(ascending=False) \
    .head(1)

,total_amount
cuisine,
Mexican,808.021344


In [50]:
user_totals = final_df.groupby('user_id')['total_amount'].sum()

(user_totals > 1000).sum()

np.int64(2544)

In [51]:
#revenue by restaurant rating range
final_df['rating_range'] = pd.cut(
    final_df['rating'],
    bins=[0, 2, 3, 4, 5],
    labels=['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0']
)

final_df.groupby('rating_range')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)

/tmp/ipython-input-116173977.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby('rating_range')['total_amount'] \


,total_amount
rating_range,
4.6-5.0,4157357.01
4.1-4.5,3599248.98
3.6-4.0,255018.13
3.0-3.5,0.00


In [52]:
#avg order value per city for gold members
final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .mean() \
    .sort_values(ascending=False) \
    .head(1)

,total_amount
city,
Chennai,808.45908


In [53]:
cuisine_stats = final_df.groupby('cuisine').agg(
    distinct_restaurants=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
)

cuisine_stats.sort_values(
    ['distinct_restaurants', 'total_revenue'],
    ascending=[True, False]
).head()

,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [54]:

gold_orders = final_df[final_df['membership'] == 'Gold']['order_id'].count()
total_orders = final_df['order_id'].count()

round((gold_orders / total_orders) * 100)

50

In [55]:
#highest aov
restaurant_stats = final_df.groupby('restaurant_name_x').agg(
    avg_order_value=('total_amount', 'mean'),
    total_orders=('order_id', 'count')
)

restaurant_stats[
    restaurant_stats['total_orders'] < 20
].sort_values('avg_order_value', ascending=False).head()


,avg_order_value,total_orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19


In [56]:
#highest revenue by membership and cuisine
final_df.groupby(['membership', 'cuisine'])['total_amount'] \
    .sum() \
    .sort_values(ascending=False) \
    .head()

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Name: total_amount, dtype: float64

In [57]:
#total orders by gold members
gold_orders_count = final_df[final_df['membership'] == 'Gold']['order_id'].count()
gold_orders_count


np.int64(4987)

In [58]:
#total revenue from hyd
hyderabad_revenue = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
round(hyderabad_revenue)

1889367

In [59]:
#no.of distinct users
distinct_users_count = final_df['user_id'].nunique()
distinct_users_count


2883

In [60]:
#aov for gold members
avg_gold_aov = round(final_df[final_df['membership'] == 'Gold']['total_amount'].mean(), 2)
avg_gold_aov


np.float64(797.15)

In [61]:
high_rating_orders = final_df[final_df['rating'] >= 4.5]['order_id'].count()
high_rating_orders


np.int64(3374)

In [62]:
#orders in top revenue city
gold_city_revenue = final_df[final_df['membership'] == 'Gold'] \
    .groupby('city')['total_amount'].sum()

top_gold_city = gold_city_revenue.idxmax()
top_gold_city

orders_top_gold_city = final_df[(final_df['membership'] == 'Gold') & (final_df['city'] == top_gold_city)]['order_id'].count()
orders_top_gold_city


np.int64(1337)

In [63]:

total_rows = final_df.shape[0]
print("Total number of rows in the final merged dataset:", total_rows)


Total number of rows in the final merged dataset: 10000
